## Install and import dependencies

In [3]:
%pip install torch gensim datasets nltk 
%pip setuptools wheel
%pip install -U spacy

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mythilimulani/Projects/SC4002-NLP-Group-Project/myenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/var/folders/rw/hx2p35lj2fb3nx3xpvyr3b940000gn/T/ipykernel_45773/2373811666.py", line 1, in <module>
    get_ipython().run_line_magic('pip', 'install torch gensim datasets nltk')
  File "/Users/mythilimulani/Projects/SC4002-NLP-Group-Project/myenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2480, in run_line_magic
  File "/Users/mythilimulani/Projects/SC4002-NLP-Group-Project/myenv/lib/python3.12/site-packages/IPython/core/magics/packaging.py", line 105, in pip
  File "/Users/mythilimulani/Projects/SC4002-NLP-Group-Project/myenv/lib/python3.12/site-packages/ipykernel/zmqshell.py", line 657, in system_piped
  File "/Users/mythilimulani/Projects/SC4002-NLP-Group-Project/myenv/lib/python3.12/site-packages/IPython/utils/_process_posix.py", line 125, in system
ModuleNotF

In [2]:
import os
# import nltk
# nltk.download("all")


import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import gensim.downloader as api
import spacy
import random
import pickle

from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import accuracy_score

from datasets import load_dataset
from gensim.models import KeyedVectors



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mythilimulani/Projects/SC4002-NLP-Group-Project/myenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/var/folders/rw/hx2p35lj2fb3nx3xpvyr3b940000gn/T/ipykernel_45773/1038347899.py", line 6, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mythilimulani/Projects/SC4002-NLP-Group-Project/myenv/lib/python3.12/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mythilimulani/Projects/SC4002-NLP-Group-Project/myenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
  File "/Users/mythilimulani/Projects/SC4002-N

## Part 0. Dataset Preparation

In [ ]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train'] 
validation_dataset = dataset['validation']
test_dataset = dataset['test']

In [ ]:
#Number of sentences in each set 
print(f"Size of training set: {train_dataset.num_rows} sentences")
print(f"Size of validation set: {validation_dataset.num_rows} sentences")
print(f"Size of test set: {test_dataset.num_rows} sentences")

Size of training set: 8530 sentences
Size of validation set: 1066 sentences
Size of test set: 1066 sentences


In [ ]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load the embedding matrix and word_to_index from the pickle file
with open("updated_embedding_matrix.pkl", "rb") as f:
    data = pickle.load(f)
    embedding_matrix = data["embeddings"]
    word_to_index = data["word_to_index"]

# Convert embedding_matrix to a NumPy array and a PyTorch tensor
embedding_matrix_array = np.array(embedding_matrix)
embedding_matrix_tensor = torch.tensor(embedding_matrix_array, dtype=torch.float32)

print(f"Loaded embedding matrix with shape: {embedding_matrix_array.shape}")
print(f"Vocabulary size (word_to_index): {len(word_to_index)}")

tokenise train, validation, test dataset

In [ ]:
pre_tokenized_train_texts = []
for sentence in train_dataset['text']:
    # Tokenize the sentence using spaCy and store tokens as a list of strings
    tokens = [token.text for token in nlp(sentence.lower())]
    pre_tokenized_train_texts.append(tokens)

In [ ]:
# Pre-tokenize validation and test sets
pre_tokenized_validation_texts = [[token.text for token in nlp(sentence.lower())] for sentence in validation_dataset['text']]
pre_tokenized_test_texts = [[token.text for token in nlp(sentence.lower())] for sentence in test_dataset['text']]


## Part 3.3.Keeping the above two adjustments, replace your simple RNN model in Part 2 wioth a biLSTM model and biGRU model.

# biLSTM Model

preparing the train dataset. text -> word index

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True  # Ensures reproducibility in CUDA operations
    torch.backends.cudnn.benchmark = False     # Disables some optimizations to ensure determinism

# Set the seed
set_seed(42)

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, tokenized_texts, labels : list[int], vocab : set, embedding_matrix : dict, max_len=30):
        self.texts = tokenized_texts
        self.labels  = labels
        self.vocab = word_to_index
        self.embedding_matrix : dict = embedding_matrix
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        vectorized_text = self.vectorize(text)
        return torch.tensor(vectorized_text, dtype=torch.long), torch.tensor(label, dtype=torch.float)
            
    def vectorize(self, tokens):
        # Convert tokens to their corresponding index in the vocabulary
        vectorized = [self.vocab.get(token, self.vocab['<UNK>']) for token in tokens]

        # Pad or truncate to max_len
        if len(vectorized) < self.max_len:
            vectorized += [self.vocab['<PAD>']] * (self.max_len - len(vectorized))
        else:
            vectorized = vectorized[:self.max_len]
        return vectorized

    def build_vocab_dict(self, vocab : set):
        if "<PAD>" in vocab : vocab.remove("<PAD>")
        if "<UNK>" in vocab : vocab.remove("<UNK>")
        vocab_dict = {word: idx for idx, word in enumerate(vocab)}
        vocab_dict['<PAD>'] = len(vocab_dict) # Add padding token
        vocab_dict['<UNK>'] = len(vocab_dict) # Add unknown token
        return vocab_dict


make the bilstm

In [ ]:
class SentimentBiLSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, dropout_rate):
        super(SentimentBiLSTM, self).__init__()
        # Load pre-trained embeddings
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_matrix.size(1), hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        # print("LSTM out: ", lstm_out.shape)
        # Get the final forward and backward hidden states
        out = torch.cat((lstm_out[:, -1, :self.lstm.hidden_size], lstm_out[:, 0, self.lstm.hidden_size:]), dim=1)
        # print(out)
        out = self.dropout(out)
        # return self.sigmoid(self.fc(out))
        return self.fc(out)

### Training, evaluation functions

training function

In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in iterator:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move to GPU
        optimizer.zero_grad()
        output = model(X_batch).squeeze(1)
        loss = criterion(output, y_batch.float())
        loss.backward()
        # for param in model.parameters():
        #     if param.grad is not None:  # Ensure the gradient is not None
        #         print(f"Gradient norm for {param.shape}: {param.grad.data.norm()}")
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)


evaluation function

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in iterator:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move to GPU
            output = model(X_batch).squeeze(1)
            
            probs = model.sigmoid(output)
            prediction = (probs >= 0.5).float()
            
            loss = criterion(output, y_batch.float())

            epoch_loss += loss.item()
            all_preds.extend(prediction.tolist())
            all_labels.extend(y_batch.tolist())
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy, epoch_loss / len(iterator)


train and validate function

In [ ]:
def train_and_validate(num_epochs, model, train_iterator, valid_iterator, optimizer, criterion, scheduler):
    for epoch in range(num_epochs):
        train_loss = train(model, train_iterator, optimizer, criterion)
        accuracy , valid_loss = evaluate(model, valid_iterator, criterion)
        scheduler.step(valid_loss)
        if (epoch==0):
                best_val_loss = valid_loss
                epochs_without_improvement = 0
        print(f'Epoch {epoch + 1}: Train Loss = {train_loss:.3f}, Accuracy = {accuracy:.3f}, Val Loss = {valid_loss:.3f} Learning Rate: {scheduler.optimizer.param_groups[0]["lr"]:.6f}')

        if valid_loss < best_val_loss:
            best_val_loss = valid_loss
            epochs_without_improvement = 0  # Reset counter
        else:
            epochs_without_improvement += 1
            
        # Check for convergence
        if epochs_without_improvement >= 8:  # Convergence condition (no improvement for 4 epochs)
            print("Convergence reached, stopping training.")
            break

create model and run the train loop

In [ ]:

# embedding_matrix : dict[ str , np.ndarray]= np.load("embedding_matrix.npy",allow_pickle='TRUE').item()
# embedding_matrix_values = np.array(list(embedding_matrix.values()), dtype=np.float32)
# embedding_matrix_tensor = torch.tensor(embedding_matrix_values, dtype=torch.float32)

# Create dataset instances
train_dataset = SentimentDataset(pre_tokenized_train_texts, train_dataset['label'], word_to_index, embedding_matrix)
valid_dataset = SentimentDataset(pre_tokenized_validation_texts, validation_dataset['label'], word_to_index, embedding_matrix)
test_dataset = SentimentDataset(pre_tokenized_test_texts, test_dataset['label'], word_to_index, embedding_matrix)
# Create data loaders
train_iterator = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_iterator = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_iterator = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
hidden_dim = 128  # Adjust as needed
output_dim = 1  # Binary sentiment classification
model = SentimentBiLSTM(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate=0.0).to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4)
# Now you can run your training loop
train_and_validate(25, model, train_iterator, valid_iterator, optimizer, criterion, scheduler)

test

## Hyper-parameter Training

In [ ]:
# import itertools

# # Define the hyper-parameter grid
# hidden_dims = [64, 128 ,256]
# learning_rates = [0.001, 0.005]
# dropout_rates = [0.3, 0.5]
# batch_sizes = [32, 64]
# output_dim = 1 

# # Iterate over all combinations of hyper-parameters
# for hidden_dim, lr, dropout_rate, bs in itertools.product(hidden_dims, learning_rates, dropout_rates, batch_sizes):
#     print(f'Training with hidden_dim={hidden_dim}, lr={lr}, dropout_rate={dropout_rate}, batch_size={bs}')
    
#     model = SentimentBiLSTM(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate).to(device)
#     criterion = nn.BCEWithLogitsLoss()
#     optimizer = optim.Adam(model.parameters(), lr=lr)

#     train_iterator = DataLoader(train_dataset, bs)
#     valid_iterator = DataLoader(valid_dataset, bs)

#     train_and_validate(25, model, train_iterator, valid_iterator, optimizer, criterion)


## Training with best parameters : 
hidden_dim=64, 

lr=0.001, 

dropout_rate=0.3, 

batch_size=64

In [ ]:
hidden_dim = 64
lr=0.001
dropout_rate=0.3
bs=64
output_dim = 1 

model = SentimentBiLSTM(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4)

train_iterator = DataLoader(train_dataset, bs)
valid_iterator = DataLoader(valid_dataset, bs)

train_and_validate(25, model, train_iterator, valid_iterator, optimizer, criterion, scheduler)


NameError: name 'SentimentBiLSTM' is not defined

## Testing

In [31]:
# Converting the Vocab set to dictionary 
def build_vocab_dict(vocab_set):
    # Create the vocabulary dictionary without <PAD> and <UNK>
    vocab_set.discard("<PAD>")
    vocab_set.discard("<UNK>")
    vocab_dict = {word: idx for idx, word in enumerate(vocab_set, start=2)}

    # Check for <PAD> and <UNK> existence and assign them fixed indices if they are present
    if "<PAD>" not in vocab_dict:
        vocab_dict["<PAD>"] = 0  # Index for padding token
    if "<UNK>" not in vocab_dict:
        vocab_dict["<UNK>"] = 1  # Index for unknown token
    
    #add the <PAD> and <UNK> back to the vocab
    vocab_set.add("<PAD>")
    vocab_set.add("<UNK>")
    return vocab_dict


In [ ]:
# Step 8: Get a sample sentence from the test set and predict
import random
# Select a random index from the test dataset
random_index = random.randint(0, len(test_dataset) - 1)

# Get the corresponding sentence and its label from the test dataset
sample_sentence = test_dataset[random_index]['text']  # Assuming the dataset contains a 'text' field
true_label = test_dataset[random_index]['label']  # Assuming there's a label field
# Tokenize the sample sentence
sample_tokens = nlp(sample_sentence.lower())

vocab = word_to_index
# Convert tokens to indices
sample_indices = pre_tokenized_test_texts[random_index]
sample_tensor = torch.tensor(sample_indices).unsqueeze(0)  # Add batch dimension
sample_tensor = sample_tensor.to(device)  # Move to GPU if available
# Make prediction using the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to compute gradients during inference
    output = model(sample_tensor)  # Pass the tensor to the model
    print(output)
    probs = model.sigmoid(output)
    predicted = (probs >= 0.5)
    print(predicted.item())

# Map predicted index to sentiment label
sentiment_labels = ['negative', 'positive']  # Adjust according to your label encoding
predicted_label = sentiment_labels[predicted]

# Print results
print(f"Sample Sentence: '{sample_sentence}'")
print(f"True Label: {sentiment_labels[true_label]}")
print(f"Predicted Label: {predicted_label}")

NameError: name 'model_33' is not defined

In [ ]:
def test(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in iterator:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move to GPU
            output = model(X_batch).squeeze(1)
            
            probs = model.sigmoid(output)
            prediction = (probs >= 0.5).float()
            
            loss = criterion(output, y_batch.float())

            epoch_loss += loss.item()
            all_preds.extend(prediction.tolist())
            all_labels.extend(y_batch.tolist())
            
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy = {accuracy:.3f}")



In [ ]:
test(model, test_iterator, criterion)

Accuracy = 0.744


## BiLSTM with an Attention Layer

In [14]:
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.attn = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, hidden, encoder_outputs):
        """
        hidden: [batch_size, hidden_dim] (last hidden state of the decoder or a step in the sequence)
        encoder_outputs: [batch_size, seq_len, hidden_dim] (outputs from the encoder)
        """
        # Compute the attention weights using the query (hidden state) and keys (encoder outputs)
        attn_weights = torch.matmul(encoder_outputs, hidden.unsqueeze(2)).squeeze(2)
        
        # Apply softmax to get the attention weights
        attn_weights = F.softmax(attn_weights, dim=1)
        
        # Compute the weighted sum of the encoder outputs (values)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        
        return context, 



In [15]:
class BiLSTMWithAttention(nn.Module):
    
    
    def __init__(self, embedding_matrix, hidden_dim, output_dim, dropout_rate):
        super(BiLSTMWithAttention, self).__init__()
        # Load pre-trained embeddings
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_matrix.size(1), hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.attn = Attention(hidden_dim * 2)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x):
        # Embed the input
        embedded = self.embedding(x) #[B, S, E]
        
        # Get the outputs and hidden states from the LSTM
        lstm_out,_= self.lstm(embedded)  # lstm_out: [batch_size, seq_len, hidden_dim * 2]
        
        # Use the last hidden state of the LSTM as the query for the attention mechanism
        last_hidden = lstm_out[:, -1, :] #[B, 2 * H]
        
        # Apply attention to the LSTM outputs
        context, attn_weights = self.attn(last_hidden, lstm_out) #[B, 2 * H], [B, S]
        
        # Pass the context vector through a fully connected layer
        output = self.fc(context) #[B, output_dim]
        
        return output


In [24]:
hidden_dim = 128
lr=0.001
dropout_rate=0.3
bs=64
output_dim = 1 


model_attn= BiLSTMWithAttention(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate ).to(device)
criterion_attn = nn.BCEWithLogitsLoss()
optimizer_attn = torch.optim.Adam(model_attn.parameters(), lr=1e-3)
scheduler_attn = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_attn, mode='min', factor=0.1, patience=4)

In [ ]:
train_and_validate(2, model_attn, train_iterator,valid_iterator, optimizer_attn, criterion_attn, scheduler_attn)

Epoch 1: Train Loss = 0.629, Accuracy = 0.738, Val Loss = 0.532 Learning Rate: 0.001000
Epoch 2: Train Loss = 0.330, Accuracy = 0.748, Val Loss = 0.558 Learning Rate: 0.001000


In [ ]:
test(model_attn, test_iterator, criterion_attn)

Accuracy = 0.774


In [ ]:
train_and_validate(2, model_attn, train_iterator,valid_iterator, optimizer_attn, criterion_attn, scheduler_attn)
test(model_attn, test_iterator, criterion_attn)

Epoch 1: Train Loss = 0.012, Accuracy = 0.743, Val Loss = 1.345 Learning Rate: 0.000100
Epoch 2: Train Loss = 0.009, Accuracy = 0.741, Val Loss = 1.395 Learning Rate: 0.000100
Accuracy = 0.776
